<a href="https://colab.research.google.com/github/lavande-epices/diplome/blob/master/rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas shapely

In [ ]:
import geopandas as gpd

In [ ]:
from shapely.geometry import LineString, Polygon

In [ ]:
def priorities_speed(geojson_file, output_file=None):
    gdf = gpd.read_file(geojson_file)

    if 'fid' not in gdf.columns or 'time' not in gdf.columns:
        raise ValueError("GeoJSON должен содержать столбцы 'fid' и 'time'.")

    gdf = gdf.sort_values(by='time')
    gdf['score'] = range(1, len(gdf) + 1)
    if output_file:
        gdf.to_file(output_file, driver='GeoJSON')

    return gdf

In [ ]:
def priorities_lighting(routes_geojson, lighting_geojson):
    routes = gpd.read_file(routes_geojson)
    lighting = gpd.read_file(lighting_geojson)
о
    if routes.crs is not None and lighting.crs is not None:

        lighting = lighting.to_crs(epsg=3395)
        routes = routes.to_crs(epsg=3395)

    routes['score'] = 0
    routes['lighting_length'] = 0

    if lighting.empty:
        return priorities_speed(routes)


    lighting_lengths = {fid: 0 for fid in routes['fid']}


    for _, route in routes.iterrows():
        route_fid = route['fid']
        lighting_for_route = lighting[lighting['fid'] == route_fid]

        if not lighting_for_route.empty:
            lighting_length = lighting_for_route.geometry.length.sum()
            lighting_lengths[route_fid] = lighting_length
            routes.loc[routes['fid'] == route_fid, 'lighting_length'] = lighting_length

    best_routes = sorted(lighting_lengths.items(), key=lambda item: item[1], reverse=True)

    for rank, (fid, length) in enumerate(best_routes, start=1):
        routes.loc[routes['fid'] == fid, 'score'] = rank

    return routes

In [ ]:
routes_geojson = 'line_2.geojson'
lighting_geojson = 'lighting_2.geojson'
best_routes = calculate_best_route(routes_geojson, lighting_geojson)


<ipython-input-41-be6c15b0e9c1>:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '275.95520593153503' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  routes.loc[routes['fid'] == route_fid, 'lighting_length'] = lighting_length


In [ ]:
best_routes

,fid,time,priority_2GIS,rating,geometry,score,lighting_length
0,1,22,2,None,"LINESTRING (3386683.637 8293155.516, 3386683.1...",2,275.955206
1,2,20,1,None,"LINESTRING (3386682.744 8293155.261, 3386683.7...",1,619.449723


In [ ]:
best_routes.to_file('best_routes.geojson', driver='GeoJSON')